* People: 
    * Luke Bornn: Assistant Professor of Statistics at Simon Fraser University and a Visiting Scholar at Harvard University
    * Luke Miratrix: Assistant Professor at GSE, was assistant professor in our department until last year
    * Zach: you know Zach
* My talk in one slide:
    * take Zach's talk
        * the treatment assignment is dictated by a single covariate
        * that divides the data into two groups
        * we then fit a GP to each group
        * extrapolate to the boundary from both sides
        * take the difference of the two extrapolations to get our estimate of the treatment effect
    * we look at the spatial version of this setting
    * where geography is dictating the treatment assignment
    * data is now distributed geographically
    * treatment gets applied to a certain region
    * the boundary is a line rather than a point
    * again, we can fit a GP on either side, extrapolate to the boundary, and subtract
    * but instead of getting a scalar, that gives us a one-dimensional GP, which I call the cliff face
* Quick example of what we've seen in the literature
    * map of the Penn campus in Philadelphia
    * the Penn campus is patrolled by a private university police force
    * the researchers want to know whether the presence of that force reduces crime
    * the units are the city blocks, the outcome is the number of crimes in that block
    * it's a 2-dimensional problem, but they treat it as 1-dimensional, by looking at crime vs. distance from the boundary
    * once they've done that, they can use classical RDD methodologies
    * the other approach to this we've seen in the literature is matching units across the boundary
    * we think we can do better by treating it as a spatial problem explicitly
* Setup
    * setup: a treatment gets applied to a certain region
    * and this region neighbors another region where the treatment isn't applied: a control population
    * can think of two potential outcome surfaces on the entire space
        * but only $Y_T$ gets observed in the treatment region
        * and only $Y_C$ in the control region
* We explored two possible models
    * the first is what we call the **1GP solution**
    * we model the potential outcomes as a **single Gaussian process**
    * but with a **constant treatment effect** added to the units in the treatment region
    * very **simple** model to understand, but it suffers from a few **flaws**
    * first, the **constant treatment effect** assumption is really strong
    * second, the Gaussian process is smooth, and so the **gradient** must be the same on both sides of the discontinuity
    * and thirdly, we discovered that this model **doesn't just use information near the boundary**
    * after all, the average in the control group is $\mu$, but $\mu + \tau$, so given enough data however far away from the boundary, the inference $\tau$ will converge towards the difference in means
    * the fact that data far away from the boundary can be so informative about the treatment effect is a **violation of the philosophy of regression discontinuity designs**
* This led us to the 2GP solution
    * where we **fit a Gaussian Process on both sides** of the boundary
    * which is the 2-dimensional **analog of the methodology Zach presented**
    * the treatment effect is no longer an **explicit part of the model**
    * instead it is the difference between the two surfaces
    * theoretically, the treatment effect is defined everywhere, but just like in the 1D setting, we're not in a position to estimate it everywhere
    * so we focus on the boundary
* Just to make the procedure explicit
    * we take the data in the treatment and control regions
    * and **separately** fit GPs to them
    * we **extrapolate** both GPs to the boundary, obtaining predictive means and covariance
    * we then take the **difference** of the two extrapolations to obtain an estimate of the treatment effect along the boundary: what I call the **“cliff-face estimate”**
    * interestingly, this treatment effect cliff-face **is itself a one-dimensional Gaussian process**
    * computationally, we estimate it at a set of points along the boundary, which we call **sentinel units**
* Example: **house prices** in NYC
    * downloaded a year's worth of sales data in NYC
    * NYC is divided into several school districts
    * children who live within a school district are guaranteed attendance in a school within their district, otherwise they have to apply
    * some of these districts perform much better than others
    * **question**: to the school districts drive house prices?
    * in particular, moving across school district lines, do we observe a discontinuous jump in house prices?
    * *stare at heat map*
    * we'll focus on districts 19 and 27, which look like they might have a discontinuity
    * incidentally, that's also the boundary between Brooklyn and Queens (boroughs of New York), so the causal story is murky
    * but our focus is on detecting and quantifying discontinuities, we'll leave these interesting issues aside
* Histogram
    * does suggest a difference
* Procedure
    * here we also have **covariates**: the building classes (single family home, condo, apartment building, ...)
    * we want to integrate them into our analysis
    * this is very hard to do in classical regression discontinuity designs
    * but with Gaussian processes it's easy
    * we just add a $D \gamma$ term to the model, where $D$ is a model matrix of additional covariates, and $\gamma$ is a vector of regression coefficients
    * by putting a normal prior on $\gamma$, the linear regression terms adds a component $\sigma_\gamma^2 D^\intercal D$ to the model matrix ($\rightarrow$ ridge regression)
    * but it's still a GP! so all the MVN machinery still works
    * what we do next is obtain the maximum likelihood estimator for all the **hyperparameters**: $\sigma_y$, $\sigma_{GP}$ and $\sigma_\gamma$
    * to simplify the problem, we extract the posterior mean on the $\gamma$ coefficients, using **only district 27 data** to avoid any possibility of the treatment affecting the estimates, and then subtract the linear regression term to get residuals
    * we then **fit** a Gaussian process **to the residuals** on either side
    * **extrapolate** to the boundary
    * **subtract** the predictions
    * and then **make some plots**
* So now we've got our cliff face: so what?
    * we want to ask questions about the treatment effect
    * first, what is the **average** treatment effect?
    * but what does it mean to average over the boundary?
    * amongst ourselves we had quite a debate over the best way to do this averaging
    * the most obvious way is to just take the simple mean of the estimated treatment effect at each sentinel location
* But there are issues with this simple averaging
    * I'll illustrate the first issue with the border between Louisiana and Mississippi
    * the border first follows the Mississippi river, then becomes a straight horizontal lign, then follows the Pearl river
    * let's see what happens when we add sentinels
    * more sentinels get added to the meanderous section of the border than the straight section
    * so when we do the simple average, we give more weight to wiggly sections
    * this is **troubling**: it means our estimand and estimator depend heavily on the wiggliness of the rivers, even though our treatment effect **probably has nothing to do with rivers**
* Second issue
    * this is the border between McHenry and Lake counties in Illinois
    * the border goes through a lake and then near the city of McHenry
    * the variance of the cliff-face will be very high in the lake, and much lower near the populated region
    * but the unweighted mean treats both equally
    * this problem can be resolved by taking a **density-weighted mean**, using some kind of density estimates
    * we've implemented some version of this, but it doesn't solve the first problem (wiggliness-sensitivity)
* My solution: **inverse-variance weighted mean**
    * in my view, the most natural way to summarize the information contained in the treatment effect cliff face
    * gives more weight to sections of the boundary that we have better estimates for
    * does so in a way that accounts for the correlation structure of the cliff-face
    * inverse-variance weights are also guaranteed to minimize the variance of a weighted mean: nice bonus
    * it's in the spirit of the regression-discontinuity design: we estimate the treatment effect only where we actually have information about it
    * but there's a problem: we don't know what the estimand is
    * which makes Luke Miratrix upset
    * in our NY example, we get a strong effect
* **Hypothesis testing**
    * there is also interest in answering the question: is the treatment effect non-zero anywhere
    * not necessarily answered by the average treatment effect: one section could have positive effect, and another negative
    * one can use a chi-squared test to get at this
    * but the covariance on the cliff face is often of low rank, if not mathematically, then computationally
    * Benavoli and Mangili suggest a solution to this: remove eigenvalue below a certain threshold
    * nu is the number of eigenvalues above epsilon
    * in B&M, they have simulations that indicate that this test is conservative: under the null, p-value below 0.05 *less* than 5% of the time
    * my simulations in this particular context show the same thing
    * taking a frequentist stance, I've also looked at the distribution of this test-statistic under the null-hypothesis that the outcomes come from a single GP, and found that there is an extra component to the variance which accounts for this conservatism
    * but I'm finding it difficult to reason about this test: a frequentist test that uses a Bayesian posterior as its test statistic
* Pairwise comparisons
    * going back to the NYC example
    * long at each pair of adjacent school districts
    * obtain the treatment effect cliff-face
    * compute the average treatment effect using inverse-variance weights
    * get a mean and variance
    * compute the effect size mean / standard deviation
    * plot border with that thickness